In [1]:
import cv2
import numpy as np

from ultralytics import YOLO

In [2]:
LEAF_MODEL_PATH = "models/leaf.pt"
LEAF_DISEASE_MODEL_PATH = "models/leafdisease.pt"

In [3]:
def preprocessImage(image: np.ndarray) -> np.ndarray:
    """Preprocess image

    Args:
        image (np.ndarray): Image object of numpy.ndarray type

    Returns:
        np.ndarray: Preprocessed image of numpy.ndarray type
    """
    return cv2.resize(image, (640, 640))

In [4]:
def getSegmentationMask(MODEL_PATH: str, image: np.ndarray) -> np.ndarray:
    """Extract Segmentation Mask

    Args:
        MODEL_PATH (str): Path to the segmentation model
        image (np.ndarray): Image object of numpy.ndarray type

    Returns:
        np.ndarray: Binary Segmentation mask
    """
    
    MODEL = YOLO(MODEL_PATH)
    
    result = MODEL(image)
    
    mask = result[0].masks.data
    
    try:
        binary_image = mask.numpy()
    except:
        binary_image = mask.cpu().numpy()
        
    binary_image = binary_image.astype('uint8')
    
    binary_image = np.bitwise_or.reduce(binary_image, axis = 0)
    
    return binary_image

In [5]:
image = cv2.imread("img3.jpg")
image = preprocessImage(image)
binary_mask_leaf = getSegmentationMask(LEAF_MODEL_PATH, image)
binary_mask_leaf_disease = getSegmentationMask(LEAF_DISEASE_MODEL_PATH, image)


0: 640x640 1 Leaf, 12.5ms
Speed: 21.3ms preprocess, 12.5ms inference, 378.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Disease, 22.2ms
Speed: 5.0ms preprocess, 22.2ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)


In [6]:
np.sum(binary_mask_leaf * binary_mask_leaf_disease) / np.sum(binary_mask_leaf) * 100

94.34113517387132

In [7]:
binary_mask_leaf.shape

(640, 640)

In [8]:
binary_mask_leaf_disease.shape

(640, 640)

In [9]:
cv2.imwrite('seg_leaf.jpg', binary_mask_leaf * 255)
cv2.imwrite('seg_leaf_disease.jpg', binary_mask_leaf_disease * 255)

True

In [10]:
color_mask = np.zeros_like(image)
color_mask[binary_mask_leaf_disease == 1] = [255, 0, 0]  # Green color for the mask

# Overlay the mask on the image
overlay = cv2.addWeighted(image, 1, color_mask, 0.3, 0)
output_path = 'overlay_result_leaf.jpg'
cv2.imwrite(output_path, overlay)

True